In [1]:
# from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load

# verbose = False  # True prints more output
verbose = True  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'
df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [2]:
# number of days to lookback from last df date for performance evaluation
days_lookbacks = [15, 30]
days_lookbacks.sort()  # sort list of integers in ascending order

# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  # slice start of set_most_common_syms
syms_end = 5  #  slice end of set_most_common_syms

In [3]:
# grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
# grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
grp_most_common_syms = []  # accumulates the best performing symbols from each days_lookback
grp_perf_ranks = []  # accumulates the best performance ranking symbols from each days_lookback 
grp_dates = []  # dates of days_lookbacks 

for d_lookback in days_lookbacks:
  df = df_close_clean.iloc[-d_lookback:]
  perf_ranks, most_common_syms = _5_perf_ranks(df, n_top_syms=n_top_syms)  # get the best performance symbols and rankings
  grp_most_common_syms.append(most_common_syms)
  grp_perf_ranks.append(perf_ranks)  
  grp_dates.append(df.index[0].strftime("%Y-%m-%d")) 

grp_dates.append(df.index[-1].strftime("%Y-%m-%d"))  # get df's last date
grp_dates.sort()

In [4]:
# flatten list of lists of (symbol:frequency)
flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
# group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
# get the top few most-frequent symbol:frequency pairs
top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
# get symbols from top_set_syms_n_freq
top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]

In [5]:
print(f'grp_dates: {grp_dates}')
print(f'top_set_syms_n_freq: {top_set_syms_n_freq}')
print(f'top_set_syms: {top_set_syms}')

grp_dates: ['2022-12-30', '2023-01-24', '2023-02-13']
top_set_syms_n_freq: [('CRUS', 6), ('CTLT', 6), ('THR', 5), ('ICPT', 4), ('LRN', 4), ('TSLA', 4), ('ALGN', 3), ('BSIG', 3), ('CINF', 3), ('ELF', 3), ('EVRI', 3), ('META', 3), ('ON', 3), ('RVNC', 3), ('SMG', 3), ('STBA', 3), ('STNG', 3), ('URI', 3), ('WBS', 3), ('ACLS', 2)]
top_set_syms: ['CRUS', 'CTLT', 'THR', 'ICPT', 'LRN']


In [6]:
# import webbrowser

# y_quote = 'https://finance.yahoo.com/quote/'
# for sym in top_set_syms:
#   url = y_quote + sym
#   print(url)
#   webbrowser.open(url)  # open symbols in yahoo finance

In [7]:
# # Importing Pandas to create DataFrame
# import pandas as pd

# my_cols = ['date', 'days_lookbacks', 'n_top_syms', 'syms_start', 'syms_end', 'grp_dates', 'top_set_syms_n_freq', 'top_set_syms']
# # Creating Empty DataFrame and Storing it in variable df
# df = pd.DataFrame(columns=my_cols)

In [8]:
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_top_set_syms = 'df_top_set_syms'  # df to store top set symbols

In [9]:
df = pickle_load(path_data_dump, fp_top_set_syms)
df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-02-10,"[15, 30]",20,0,5,"[2022-12-29, 2023-01-23, 2023-02-10]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ALGN, 4), (I...","[CRUS, CTLT, THR, ALGN, ICPT]"


In [10]:
row_add = [grp_dates[-1], days_lookbacks, n_top_syms, syms_start, syms_end, grp_dates, top_set_syms_n_freq, top_set_syms]
df.loc[len(df)] = row_add
df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-02-10,"[15, 30]",20,0,5,"[2022-12-29, 2023-01-23, 2023-02-10]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ALGN, 4), (I...","[CRUS, CTLT, THR, ALGN, ICPT]"
1,2023-02-13,"[15, 30]",20,0,5,"[2022-12-30, 2023-01-24, 2023-02-13]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ICPT, 4), (L...","[CRUS, CTLT, THR, ICPT, LRN]"


In [11]:
df = df.drop_duplicates(subset=['date'])

In [12]:
pickle_dump(df, path_data_dump, fp_top_set_syms)

In [13]:
_df = pickle_load(path_data_dump, fp_top_set_syms)
_df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-02-10,"[15, 30]",20,0,5,"[2022-12-29, 2023-01-23, 2023-02-10]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ALGN, 4), (I...","[CRUS, CTLT, THR, ALGN, ICPT]"
1,2023-02-13,"[15, 30]",20,0,5,"[2022-12-30, 2023-01-24, 2023-02-13]","[(CRUS, 6), (CTLT, 6), (THR, 5), (ICPT, 4), (L...","[CRUS, CTLT, THR, ICPT, LRN]"


In [14]:
import pandas as pd
# pd.options.display.width = 120
pd.options.display.max_colwidth = 120

print(f'_df["top_set_syms_n_freq"]:\n{_df["top_set_syms_n_freq"]}\n')
l_last_top_set_syms_n_freq = _df['top_set_syms_n_freq'].iloc[-1]

print(f'_df["top_set_syms"]:\n{_df["top_set_syms"]}')
l_last_top_set_syms = _df['top_set_syms'].iloc[-1]
print(f'last row of top_set_syms:\n{l_last_top_set_syms}')

_df["top_set_syms_n_freq"]:
0    [(CRUS, 6), (CTLT, 6), (THR, 5), (ALGN, 4), (ICPT, 4), (LRN, 4), (TSLA, 4), (AXP, 3), (CINF, 3), (ELF, 3), (EVRI, 3)...
1    [(CRUS, 6), (CTLT, 6), (THR, 5), (ICPT, 4), (LRN, 4), (TSLA, 4), (ALGN, 3), (BSIG, 3), (CINF, 3), (ELF, 3), (EVRI, 3...
Name: top_set_syms_n_freq, dtype: object

_df["top_set_syms"]:
0    [CRUS, CTLT, THR, ALGN, ICPT]
1     [CRUS, CTLT, THR, ICPT, LRN]
Name: top_set_syms, dtype: object
last row of top_set_syms:
['CRUS', 'CTLT', 'THR', 'ICPT', 'LRN']


In [15]:
# type(_df["top_set_syms"])

In [16]:
recent_top_set_syms = []
for symbols in _df["top_set_syms"]:
  recent_top_set_syms.append(symbols)
recent_top_set_syms

# flatten symbols_download_err which is a list-of-lists
recent_top_set_syms = [val for sublist in recent_top_set_syms for val in sublist]
recent_top_set_syms
# get list of unique symbols in df_OHLCV, list(df) is a list of tuples
# e.g.: [('AAPL', 'Open')..('AAPL', 'Volume'),...
#        ('ZZZZ', 'Open')..('ZZZZ', 'Volume')]
recent_top_set_syms = list(set(recent_top_set_syms))
recent_top_set_syms.append("SPY")
recent_top_set_syms.sort()
print(recent_top_set_syms)


['ALGN', 'CRUS', 'CTLT', 'ICPT', 'LRN', 'SPY', 'THR']


In [17]:
# my_cols.sort() will not change order of l_last_top_set_syms 
my_cols = recent_top_set_syms.copy()  
my_cols.sort()
# my_cols.append('SPY')
_df = df_close_clean[my_cols].tail(10)
# print(_df)
print(_df.pct_change())
_df.pct_change().tail(1)[0:10]

                ALGN      CRUS      CTLT      ICPT       LRN       SPY  \
Date                                                                     
2023-01-31       NaN       NaN       NaN       NaN       NaN       NaN   
2023-02-01  0.047455  0.031530  0.021849  0.027382  0.007454  0.010628   
2023-02-02  0.273776  0.017696  0.035088  0.020789 -0.016185  0.014557   
2023-02-03 -0.046627  0.078933 -0.010417 -0.006788  0.007051 -0.010629   
2023-02-06 -0.001865 -0.000098  0.195361  0.086225 -0.026138 -0.006111   
2023-02-07  0.007621  0.041809  0.068358  0.050823  0.001677  0.013079   
2023-02-08 -0.029675 -0.014440 -0.003772 -0.044219 -0.025359 -0.010935   
2023-02-09 -0.040170 -0.001237  0.009676  0.004337  0.015955 -0.008669   
2023-02-10 -0.017332 -0.004572 -0.021389  0.000960  0.005074  0.002334   
2023-02-13  0.029923  0.015694  0.001277 -0.011026  0.025481  0.011739   

                 THR  
Date                  
2023-01-31       NaN  
2023-02-01  0.001298  
2023-02-02  0.00518

,ALGN,CRUS,CTLT,ICPT,LRN,SPY,THR
Date,,,,,,,
2023-02-13,0.029923,0.015694,0.001277,-0.011026,0.025481,0.011739,0.006294


In [18]:

_df_chg = _df.pct_change().tail(4)
print(f'_df_chg.mean(): {_df_chg.mean()}')
print(f'_df_chg.std(): {_df_chg.std()}')

_df_chg.mean(): ALGN   -0.014313
CRUS   -0.001139
CTLT   -0.003552
ICPT   -0.012487
LRN     0.005288
SPY    -0.001383
THR     0.008512
dtype: float64
_df_chg.std(): ALGN    0.030933
CRUS    0.012544
CTLT    0.013121
ICPT    0.022158
LRN     0.022067
SPY     0.010493
THR     0.020329
dtype: float64


In [19]:
print(_df_chg.mean()[:-1].mean())
_df_chg.mean()[:-1]

-0.004597697558701194


ALGN   -0.014313
CRUS   -0.001139
CTLT   -0.003552
ICPT   -0.012487
LRN     0.005288
SPY    -0.001383
dtype: float64